# DSO 510 Project: Data Collection

## Importing Libaries 

In [35]:
import matplotlib.pyplot as plt # plotting
#import basemap # plotting maps
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3 # read the database

pd.set_option("display.max_columns", None)

## Finding the SWITRS Database

In [36]:
for dirname, _, filenames in os.walk(' '): #your own files
    for filename in filenames:
        print(os.path.join(dirname, filename))

## SQL Query to extract our raw data

We are selecting only the variables that we think might be useful for our analysis at this stage. 


We are joinning the `parties` table to `collisions` table by matching their `case_id`. 


We filtered for the vehicle makes that we are interested in and passengar cars and sport utility vehicle car types. 

In [37]:
with sqlite3.connect(" ") as con: #your own files

    query = (
        '''
        SELECT 
            p.case_id, vehicle_year, vehicle_make, party_age, party_sex, 
            statewide_vehicle_type, chp_vehicle_type_towing, party_type, 
            at_fault, party_sobriety, party_number_killed, 
            party_number_injured, collision_severity, weather_1, lighting,
            collision_date, county_location
        FROM parties p 
        LEFT JOIN collisions c 
        ON p.case_id = c.case_id
        WHERE 
            vehicle_make IN ('mercedes-benz', 'bmw', 'audi', 'volvo', 'porsche', 'volkswagen') AND 
            statewide_vehicle_type == 'passenger car' AND 
            chp_vehicle_type_towing IN ('passenger car, station', 'sport utility vehicle')
        ;'''
    )

    # Construct a Dataframe from the results
    df = pd.read_sql_query(query, con)

In [38]:
df.shape

(950533, 17)

In [39]:
# Check if the initial query was done properly. 
df.head(3)

,case_id,vehicle_year,vehicle_make,party_age,party_sex,statewide_vehicle_type,chp_vehicle_type_towing,party_type,at_fault,party_sobriety,party_number_killed,party_number_injured,collision_severity,weather_1,lighting,collision_date,county_location
0,3899446,2004.0,bmw,36.0,male,passenger car,"passenger car, station",driver,1,had not been drinking,0,0,property damage only,cloudy,dusk or dawn,2009-02-11,los angeles
1,3899454,2001.0,volkswagen,51.0,female,passenger car,"passenger car, station",driver,0,had not been drinking,0,0,pain,cloudy,dark with street lights,2009-02-08,los angeles
2,3899457,2004.0,volvo,34.0,female,passenger car,"passenger car, station",driver,1,had not been drinking,0,0,property damage only,clear,daylight,2009-02-10,los angeles


## Data Cleaning 

### Taking care of missing values

In [40]:
# drop rows with missing values on these columns because we don't know what do map it to
df = df.dropna(subset=['vehicle_year','chp_vehicle_type_towing', 'party_type', 'weather_1', 'lighting', 'party_sex', 'party_age'])

# We will assume that if sobriety was not reported, then the driver was sober
df['party_sobriety'] = df['party_sobriety'].fillna('had not been drinking')

# check that we don't have any missing values 
df.isna().value_counts()

case_id  vehicle_year  vehicle_make  party_age  party_sex  statewide_vehicle_type  chp_vehicle_type_towing  party_type  at_fault  party_sobriety  party_number_killed  party_number_injured  collision_severity  weather_1  lighting  collision_date  county_location
False    False         False         False      False      False                   False                    False       False     False           False                False                 False               False      False     False           False              853548
dtype: int64

In [41]:
df.shape

(853548, 17)

## Create dummy variable for `collision_severity `
- We will map **Fatal** and **Severe Injuries** to 1 and others to 0. 

In [42]:
severity_map = {
        'fatal': 1,
        'severe injury': 1,
        'property damage only': 0,
        'pain': 0,
        'other injury': 0
}
df['severity'] = df['collision_severity'].map(severity_map)

df['severity'].mean() # about 2.1% of the collisons end in fatal or severe injury. 

0.021507870676282997

## Create dummy variable for `party_sobriety`
- Map **had been drinking, under influence** to 1 and others to 0.

In [43]:
list_0 = df['party_sobriety'].unique()[df['party_sobriety'].unique() != 'had been drinking, under influence'] 
sobriety_map = {'had been drinking, under influence': 1}
for i in list_0: 
    sobriety_map[i] = 0
df['sobriety'] = df['party_sobriety'].map(sobriety_map)
df['sobriety'].mean() # about 4.3% of the collisions involved DUI 

0.04690655944363996

## Create dummy variable for `vehicle_make`
- Map **volvo** to 1 and others to 0. 

In [44]:
not_volvo = df['vehicle_make'].unique()[df['vehicle_make'].unique() != 'volvo']
volvo_map = {'volvo':1}
for i in not_volvo: 
    volvo_map[i] = 0 
df['volvo'] = df['vehicle_make'].map(volvo_map)
df.head()

,case_id,vehicle_year,vehicle_make,party_age,party_sex,statewide_vehicle_type,chp_vehicle_type_towing,party_type,at_fault,party_sobriety,party_number_killed,party_number_injured,collision_severity,weather_1,lighting,collision_date,county_location,severity,sobriety,volvo
0,3899446,2004.0,bmw,36.0,male,passenger car,"passenger car, station",driver,1,had not been drinking,0,0,property damage only,cloudy,dusk or dawn,2009-02-11,los angeles,0,0,0
1,3899454,2001.0,volkswagen,51.0,female,passenger car,"passenger car, station",driver,0,had not been drinking,0,0,pain,cloudy,dark with street lights,2009-02-08,los angeles,0,0,0
2,3899457,2004.0,volvo,34.0,female,passenger car,"passenger car, station",driver,1,had not been drinking,0,0,property damage only,clear,daylight,2009-02-10,los angeles,0,0,1
3,3899497,2003.0,mercedes-benz,43.0,male,passenger car,"passenger car, station",driver,0,had not been drinking,0,0,pain,clear,dark with street lights,2009-02-25,los angeles,0,0,0
4,3899497,2003.0,mercedes-benz,55.0,male,passenger car,"passenger car, station",driver,0,had not been drinking,0,1,pain,clear,dark with street lights,2009-02-25,los angeles,0,0,0


## Create categorical variable for `weather_1` and `lighting
- Assign number for each weather condition and lighting.
- The mappings printed below

In [45]:
df['weather_1'].value_counts()

# drop weather condition marked as other since we don't know if it was good or bad weather 
df.drop(df[df['weather_1']== 'other'].index, inplace = True)
df['weather_1'].value_counts()

clear      711082
cloudy     113339
raining     25588
fog          2209
snowing       667
wind          235
Name: weather_1, dtype: int64

In [46]:
weather_map = dict((condition, 0) if condition in ['clear','cloudy'] else (condition,1) for condition in df['weather_1'].unique())

weather_map
df['weather'] = df['weather_1'].map(weather_map)

## Create categorical variable for lighting

In [47]:
lighting_map = dict((condition, 0) if condition in ['dark with street lights','daylight'] else (condition,1) for condition in df['lighting'].unique())
df['light'] = df['lighting'].map(lighting_map)

## Create dummy variable for `vehicle_type`

In [48]:
df['suv'] = df['chp_vehicle_type_towing'].map({'passenger car, station': 0, 'sport utility vehicle':1})

## Create `vehicle_age` variable 
- Calculate the age of the vehicle at the time of the collision 

In [49]:
df['collision_date'] = pd.to_datetime(df['collision_date'])

df['vehicle_age'] = df['collision_date'].dt.year - df['vehicle_year']

df.head(3)

,case_id,vehicle_year,vehicle_make,party_age,party_sex,statewide_vehicle_type,chp_vehicle_type_towing,party_type,at_fault,party_sobriety,party_number_killed,party_number_injured,collision_severity,weather_1,lighting,collision_date,county_location,severity,sobriety,volvo,weather,light,suv,vehicle_age
0,3899446,2004.0,bmw,36.0,male,passenger car,"passenger car, station",driver,1,had not been drinking,0,0,property damage only,cloudy,dusk or dawn,2009-02-11,los angeles,0,0,0,0,1,0,5.0
1,3899454,2001.0,volkswagen,51.0,female,passenger car,"passenger car, station",driver,0,had not been drinking,0,0,pain,cloudy,dark with street lights,2009-02-08,los angeles,0,0,0,0,0,0,8.0
2,3899457,2004.0,volvo,34.0,female,passenger car,"passenger car, station",driver,1,had not been drinking,0,0,property damage only,clear,daylight,2009-02-10,los angeles,0,0,1,0,0,0,5.0


### Create dummy variable for `party_sex`

In [50]:
print(df['party_sex'].value_counts())

# the sex category of X was introduced very recently so there was only 9 incidences so we drop.
df.drop(df[df['party_sex']== 'X'].index, inplace = True)

df['party_sex'] = df['party_sex'].map({'male':0, 'female':1})

male      458271
female    394840
X              9
Name: party_sex, dtype: int64


In [51]:
print(df['party_type'].value_counts())
# we will only consider cases where the party involved in the vehicle was the driver 
df.drop(df[df['party_type'] == 'parked vehicle'].index, inplace=True)
df.drop(df[df['party_type'] == 'other'].index, inplace=True)

driver            852990
other                 63
parked vehicle        58
Name: party_type, dtype: int64


## Exploratory Data Analysis 

In [52]:
df.describe()

,vehicle_year,party_age,party_sex,at_fault,party_number_killed,party_number_injured,severity,sobriety,volvo,weather,light,suv,vehicle_age
count,852990.000000,852990.000000,852990.000000,852990.000000,852990.000000,852990.000000,852990.000000,852990.000000,852990.000000,852990.000000,852990.000000,852990.000000,852990.000000
mean,2003.804333,38.683788,0.462854,0.433526,0.002598,0.293748,0.021495,0.046904,0.088001,0.033637,0.110790,0.099606,8.110940
std,8.604303,16.004956,0.498619,0.495562,0.058351,0.572966,0.145027,0.211434,0.283297,0.180293,0.313872,0.299474,6.976608
min,1907.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28.000000
25%,2000.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,2004.000000,36.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000
75%,2010.000000,50.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000
max,2048.000000,121.000000,1.000000,1.000000,5.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000


- Looking at `party_age` column, we see that the minimum party age is 0 which doesn't make sense. Therefore, we will be filtering out rows where `party_age < 16` which is the legal driving age in California. 

In [53]:
df = df.query('party_age >= 16')

- Looking at the `vehicle_age` column, we see that the minimum age of a car is -28, which doesn't make sense.
- We looked how many cars had negative vehicle age. 
- Vehicle age of -1 still makes sense because car manufacturers sometimes tend to release new model year cars earlier. Therefore we will only drop rows with `vehicle_age < -1`. 
- We will map `vehicle_age == -1` to 0. 

In [61]:
print(df.query('vehicle_age < 0')['vehicle_age'].value_counts()) 
df = df.query('vehicle_age > -2')
df['vehicle_age'] = df['vehicle_age'].replace({-1:0})

-1.0    4716
Name: vehicle_age, dtype: int64


- We will also filter for `vehicle_age <= 20` to take into account for really old vehicles that we are not interested in looking at.  

In [66]:
df = df[df['vehicle_age']<= 20]

In [67]:
df.shape

(810269, 24)

## Export the file to csv for regression

In [68]:
df.to_csv('collision_data.csv')

In [65]:
df.head()

,case_id,vehicle_year,vehicle_make,party_age,party_sex,statewide_vehicle_type,chp_vehicle_type_towing,party_type,at_fault,party_sobriety,party_number_killed,party_number_injured,collision_severity,weather_1,lighting,collision_date,county_location,severity,sobriety,volvo,weather,light,suv,vehicle_age
0,3899446,2004.0,bmw,36.0,0,passenger car,"passenger car, station",driver,1,had not been drinking,0,0,property damage only,cloudy,dusk or dawn,2009-02-11,los angeles,0,0,0,0,1,0,5.0
1,3899454,2001.0,volkswagen,51.0,1,passenger car,"passenger car, station",driver,0,had not been drinking,0,0,pain,cloudy,dark with street lights,2009-02-08,los angeles,0,0,0,0,0,0,8.0
2,3899457,2004.0,volvo,34.0,1,passenger car,"passenger car, station",driver,1,had not been drinking,0,0,property damage only,clear,daylight,2009-02-10,los angeles,0,0,1,0,0,0,5.0
3,3899497,2003.0,mercedes-benz,43.0,0,passenger car,"passenger car, station",driver,0,had not been drinking,0,0,pain,clear,dark with street lights,2009-02-25,los angeles,0,0,0,0,0,0,6.0
4,3899497,2003.0,mercedes-benz,55.0,0,passenger car,"passenger car, station",driver,0,had not been drinking,0,1,pain,clear,dark with street lights,2009-02-25,los angeles,0,0,0,0,0,0,6.0
